In [63]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [64]:
data = pd.read_csv('hillary-clinton-emails/Emails.csv')
data = data.set_index(['Id'])

In [65]:
data.ix[:2,:8]

,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink
Id,,,,,,,,
1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...
2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...


In [66]:
data.ix[:2,8:16]

,MetadataCaseNumber,MetadataDocumentClass,ExtractedSubject,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber
Id,,,,,,,,
1,F-2015-04841,HRC_Email_296,FW: Wow,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841
2,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841


In [67]:
data.ix[:2,16:]

,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
Id,,,,,
1,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
2,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...


In [75]:
contents = pd.DataFrame(data.ix[:,20])
contents.columns = ["Content"]
contents.head()
msg=contents.loc[1,"Content"]
print(msg)

UNCLASSIFIED
U.S. Department of State
Case No. F-2015-04841
Doc No. C05739545
Date: 05/13/2015
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.
RELEASE IN FULL
From: Sullivan, Jacob J <Sullivan11@state.gov>
Sent: Wednesday, September 12, 2012 10:16 AM
To:
Subject: FW: Wow
From: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi
Sent: Wednesday, September 12, 2012 10:09 AM
To: Sullivan, Jacob J
Subject: Wow
What a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me
too
UNCLASSIFIED
U.S. Department of State
Case No. F-2015-04841
Doc No. C05739545
Date: 05/13/2015
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247



In [76]:
def toDelete(line):
    if (line.find("UNCLASSIFIED")==0):
        return True
    if (line.find("CONFIDENTIAL")==0):
        return True
    if (line.find("UNCLASSIFIED")==1):
        return True
    if (line.find("From:")==0):
        return True
    if (line.find("To:")==0):
        return True
    if (line.find("Subject:")==0):
        return True
    if (line.find("Sent:")==0):
        return True
    if (line.find("For:")==0):
        return True
    if (line.find("Re:")==0):
        return True
    if (line.find("Cc:")==0):
        return True
    if (line.find("Fw:")==0):
        return True
    if (line.find("Attachments:")==0):
        return True
    if (line.find("RELEASE")==0):
        return True
    if (line.find("SENSITIVE")==0):
        return True
    if (line.find("Original Message")==0):
        return True
    if (line.find("U.S. Department of State")==0):
        return True
    if (line.find("Case No")==0):
        return True
    if (line.find("Doc No")==0):
        return True
    if (line.find("Date")==0):
        return True
    if (line.find("STATE DEPT.")==0):
        return True
    if (line.find("SUBJECT TO")==0):
        return True
    if (line.find("PART")==0):
        return True
    return False

def cleanMessage(msg):
    result = ""
    lines = msg.split("\n")
    for line in lines:
        if (not toDelete(line)):
            result=result+line+"\n";
    return result

In [81]:
contents.Content = contents.Content.apply(cleanMessage)
contents.head()
msg=contents.loc[5,"Content"]
print(msg)

B6
H <hrod17@clintonemail.com>
Friday, March 11, 2011 1:36 PM
Huma Abedin
hrc memo syria aiding libya 030311.docx
Pis print.
March 3, 2011
This memo has two parts. Part one is the report that Syria is providing air support for Qaddafi. Part two is a note
to Cody from Lord David Owen, former UK foreign secretary on his views of an increasingly complex crisis. It
seems that the situation is developing into a protracted civil war with various nations backing opposing sides
with unforeseen consequences. Under these circumstances the crucial challenge is to deprive Qaddafi of his
strategic depth—his support both financial and military.
I. Report
During the afternoon of March 3, advisers to Muammar Qaddafi stated privately that the Libyan
Leader has decided that civil war is inevitable, pitting troops and mercenary troops loyal to him
against the rebel forces gathering around Benghazi. Qaddafi is convinced that these rebels are
being supported by the United States, Western Europe and Israel.

In [79]:
for msg in contents.Content:
    sentences=[]
    emailtoken = tokenize.sent_tokenize(msg)
    sentences.extend(emailtoken)
    sid = SentimentIntensityAnalyzer()
    print(sid)
    dataneg=[]
    datapos=[]
    dataneutr=[]
    datacompound=[]
    for sentence in sentences:
        ss = sid.polarity_scores(sentence)
        print(ss)
        dataneg.append(ss['neg'])
        datapos.append(ss['pos'])
        dataneutr.append(ss['neu'])
        datacompound.append(ss['compound'])

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
#print(dataneutr)
data=[sum(np.array(dataneg)), sum(np.array(datapos)), sum(np.array(dataneutr)), sum(np.array(datacompound))]

y=len(data)
x = np.arange(y)
width = 0.35
fig = plt.figure()
ax = fig.add_subplot(111)


xTickMarks=('neg', 'pos', 'neu', 'comp')
xtickNames = ax.set_xticklabels(xTickMarks)
ax.set_xticks(x+(width/2))

rects1 = ax.bar(x,data,width)
plt.show()